# Модель ARIMA (пакет [`statsmodels`](https://www.statsmodels.org/stable/index.html))

Спецификация ARIMA(p,d,q) (ARIMA = AutoRegressive Integrated Moving Average)

$$
\begin{aligned}
	\Delta ^d y_t &= \left(\sum_{j=0}^m\alpha_j t^j\right)+\sum_{j=1}^p\phi_j\Delta^d y_{t-j}+u_t+\sum_{s=1}^q\theta_s u_{t-s} &
	u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где
* p - порядок авторегрессионной части
* d - порядок интегрирования
* q - порядок части скользящего среднего 
* $\alpha_0$ - константа (d=0) или снос (d=1)

Спецификация с использование лагового оператора

$$
\begin{aligned}
	\Phi(L)(1-L)^dy_t&=\left(\sum_{j=0}^k\alpha_j t^j\right)+\Theta(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$
где многочлены
$$
\begin{aligned}
	\Phi(z)&=1-\phi_1z-\cdots-\phi_pz^p & \Theta(z)&=1+\theta_1z+\cdots+\theta_qz^q
\end{aligned}
$$

Альтернативная спецификация (оцениваемая в `statsmodels`!)

$$
\begin{aligned}
	y_t&=\sum_{j=d}^{d+m}\delta_jt^j+\epsilon_t & \Phi(L)(1-L)^d\epsilon_t&=\Theta(L)u_t & u_t&\sim WN(0,\sigma^2)
\end{aligned}
$$

In [2]:
import numpy as np
import pandas as pd

from statsmodels.tsa.api import ARIMA
from statsmodels.stats.api import het_arch, acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_predict

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Для подгонки используем классы 
* [ARIMA](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMA.html#statsmodels.tsa.arima.model.ARIMA) (специфицированная модель)
* [ARIMAResult](https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima.model.ARIMAResults.html#statsmodels.tsa.arima.model.ARIMAResults) (подогнанная модель)

При спецификации модели нужно задать параметры `order` (порядок модели) и  `trend` (порядок тренда, если тренд включается в модель)

|d|Порядок тренда $m+d$|`trend`|`trend` (альтернативно)|
|-|:-:|:-:|:-:|
|0|-|'n'|`[0]`|
|0|0|'c'|`[1]`|
|0|1|'ct'|`[1, 1]`|
|1|-|'n'|`[0]`|
|1|1|'t'|`[0, 1]`|

## Подгонка ARIMA заданного порядка

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `y`

In [3]:
y = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARIMA(2,1,1) без сноса (без тренда) для `y`

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=(1+\theta L) u_{t-1}
$$

In [ ]:
# спецификация модели
mod = ARIMA(y, order=(2,1,1), trend='n', missing='drop')
# подгонка модели на данных
res = mod.fit()
# выводим результаты подгонки
res.summary(alpha=0.05)

## Диагностика подогнанной модели

Базовые графики

In [ ]:
res.plot_diagnostics(lags=15)

plt.show()

Тест на серийную корреляцию (Ljung-Box test). Выберем число лагов 7

In [ ]:
# корректировка степеней свободы: число оцениваемых коэффициентов = число параметров - 1 (-sigma2)
model_df = mod.k_params-1
# для тест отбрасываем первые d остатков (d=mod.k_diff)
acorr_ljungbox(res.resid[mod.k_diff:] , lags=[7], model_df=model_df)

Тест на гетероскедастичность (на ARCH-эффекты)

In [ ]:
# корректировка степеней свободы: число оцениваемых коэффициентов = число параметров - 1 (-sigma2)
model_df = mod.k_params-1
# для тест отбрасываем первые d остатков (d=mod.k_diff)
lm_stat, lm_pval, f_stat, f_pval = het_arch(res.resid[mod.k_diff:], nlags=7, ddof=model_df)

lm_stat, lm_pval

## Прогнозирование

Построим прогноз на 10 периодов

In [ ]:
forecasts = res.forecast(steps=10)
forecasts

визуализация прогноза

In [ ]:
plot_predict(res, start=len(y), end=len(y)+10, alpha=0.05)

plt.show()

In [ ]:
plt.plot(y.tail(30))
plt.plot(forecasts)

plt.show()